In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import figure
import random as rd
import copy

In [149]:
class Independent_Cascade():
    def __init__(self):
        self.g = nx.DiGraph()

    def fit(self, g):
        # fit graph with probability
        out_degree = g.out_degree(weight='None')
        max_out_degree = max(out_degree, key=lambda item:item[1])
        for e in g.edges():
            if(out_degree[e[0]] >= 10):
                # g[e[0]][e[1]]['probability'] = 1 - out_degree[e[0]]/max_out_degree[1]
                g[e[0]][e[1]]['probability'] = 1 / int(np.log(out_degree[e[0]]))
            else:
                g[e[0]][e[1]]['probability'] = 1
        self.g = g
        return g
        
    # diffusion to all possible nodes
    def diffusion_all(self, seed_nodes):
        activated_nodes = copy.deepcopy(seed_nodes)
        old_activated_nodes = seed_nodes
        activate_nums = [len(activated_nodes)]
        while(True):
            new_activated_nodes = []
            for node in self.g.nodes():
                if node in activated_nodes:
                    continue
                for neighbor in self.g.neighbors(node):
                    if neighbor in old_activated_nodes and self.g[node][neighbor]['probability'] >= rd.random():
                        new_activated_nodes.append(node)
                        break
            if len(new_activated_nodes) == 0:
                break
            old_activated_nodes = new_activated_nodes
            activated_nodes.extend(new_activated_nodes)
            activate_nums.append(len(new_activated_nodes))
        return activated_nodes, activate_nums

    # diffusion to max step
    def diffusion_step(self, seed_nodes, max_step=2):
        activated_nodes = copy.deepcopy(seed_nodes)
        old_activated_nodes = seed_nodes
        activate_nums = [len(activated_nodes)]
        for _ in range(max_step):
            new_activated_nodes = []
            for node in self.g.nodes():
                if node in activated_nodes:
                    continue
                for neighbor in self.g.neighbors(node):
                    if neighbor in old_activated_nodes and self.g[node][neighbor]['probability'] >= rd.random():
                        new_activated_nodes.append(node)
                        break
            if len(new_activated_nodes) == 0:
                break
            old_activated_nodes = new_activated_nodes
            activated_nodes.extend(new_activated_nodes)
            activate_nums.append(len(new_activated_nodes))
        return activated_nodes, activate_nums

In [29]:
# read from gml
G = nx.read_gml('PB2020.gml')

In [150]:
model = Independent_Cascade()
model.fit(G)

In [163]:
sample = rd.sample(list(G.nodes), 10)
activated_nodes, activate_nums = model.diffusion_all(sample)
print(activate_nums)

[10, 1, 255, 2, 548]
